In [1]:
message ='Automated docking workflow. Required parameter include: molecules as smiles, sdf or mol2, pdb of enzymes and box centers or if nadph as cofactor C5N will be used as center'
print(message)

Automated docking workflow. Required parameter include: molecules as smiles, sdf or mol2, pdb of enzymes and box centers or if nadph as cofactor C5N will be used as center


## Install dependencies

In [ ]:
#Change working directory accordingly and supply Slope_overview.xlsx file containing raw_data tab with all slopes in Column A and pdb files for enzymes which have been tested.

#INSTAll FIRST
""" 
conda create -n docking python=3.8
conda install -c conda-forge -c schrodinger pymol-bundle
conda install -c conda-forge biopandas tqdm rdkit scikit-learn notebook
conda install matplotlib
pip install natsort meek cirpy
conda install -c conda-forge cudatoolkit-dev

conda install pytorch pytorch-cuda=11.7 -c pytorch -c nvidia
"""


: 

## Set up the necessary Boost library

1. Download 
    * https://boostorg.jfrog.io/artifactory/main/release/1.81.0/source/boost_1_81_0.tar.bz2 
2. mv to /usr/local/ #default location
3. Extract: `tar --bzip2 -xf boost_1_81_0.tar.bz2`
4. ./bootstrap.sh
5. ./b2
   #now we want to make the path globally accessable
6. LD_LIBRARY_PATH=/usr/local/boost_1_81_0/stage/lib:${LD_LIBRARY_PATH}
7. export LD_LIBRARY_PATH

## set up cuda
The absolutely easiest way to do this is with conda
1. install anaconda or miniconda
2. create env:
   1. `conda create -n gpu`
   2. `conda install -c conda-forge cudatoolkit-dev`
3. activate env: `conda activate gpu`
4. Find the nvcc compiler and remember this path
   1. `which nvcc`
   2. If you cant find nvcc, maybe reboot or reactivate conda env

## Set up vina-gpu-cuda
1. `Git clone https://github.com/Glinttsd/Vina-GPU-CUDA.git`
2. move (mv) to folder where vina-gpu-cuda is installed
3. Change the file `Makefile` in vina-gpu-cuda foler as follows: 
   1. Change BOOST_LIB_PATH:
      1. BOOST_LIB_PATH=/usr/local/boost_1_81_0 
   2. Change NVCC path:
      1. NVCC_COMPILER=/home/cewinharhar/anaconda3/envs/docking/bin/nvcc
4. sudo make clean
5. sudo make cuda #IGNORE ALL THE RANDOM ERRORS, IF A NEW EXECUTABLE POPS UP YOU'RE GOOD TO GO
6. ./Vina-GPU --config ./input_file_example/2bm2_config.txt
7. BAM

## Install additional software

### ADFRsuite-1.0 (PDB to PDBQT for ENZYME) 
go to `https://ccsb.scripps.edu/adfr/downloads/`, 


download the (linux) one, 

make it executable with `chmod a+x ADFRsuite_Linux-x86_64_1.0_install` 

and install with `sh ADFRsuite_Linux-x86_64_1.0_install`

*Used in function:* `prepare_receptor`

### obable (converter of files)
#### dependency:

* You probably need to install **cmake** first:
   * DONT TRY TO INSTALL FROM CMD (UBUNTU)
   1. Ubuntu: go to software and install cmake

Install obable: \n
cd into extracted obable folder and follow:

1. Create a 'build' directory:

   mkdir build
   cd build

2. Configure the build system. You can specify additional build
options at this time (see below):

   cmake ..

1. Compile:

   make -j2

2. Test (optional):

   make test

3. Install:

   sudo make install

### Meeko
Meeko reads an RDKit molecule object and writes a PDBQT string (or file) for AutoDock-Vina and AutoDock-GPU.<br> It converts the docking output to RDKit molecules and SD files, without loss of bond orders.

Install in conda env:
`pip install meeko`

## Additional Information 

Vina output:
    Can have affinity or user defined

IMPORTANT:
The affinity should rather be seen as a filter for non working enzymes rather than working. 
-> low affinity (negative means substrate binds (if to strong, might be inhibiting enzyme), if positive substrate doesnt bind), high chance of non functionality

Additional information could be the hydroxilation carbon distance/angle to Iron

When docking question remains: How to process with multiple results

Vina-seed: Makes big difference in result
exhaustivness == patients: when to stop (now set on 8)

SUBSTRATE 3D:
SMILES to 3D with rdkit

#KEEP IN MIND: VINA EXISTS WITH FLEXIBLE DOCKING!

#VINA OUTPUT: ONLY PBDQT file of substrate with the same coordinate system as the enzyme pdqt file

In [1]:

working_dir = r'/home/cewinharhar/GITHUB/alphaKGDGenerator/alphaFill' #working director containing pdbs 

ligand_files = '' #file containing all ligands as smiles or sdf file

slopes_file = '' #file containing all slopes
excel_sheet = '' #sheet containing slopes

mol2_files = ['substrate.mol'] #file containing all ligands as mol2 file make sure ligand_files is empty

#Set up docking parameters

class config:
    def __init__(self, 
                 working_dir, mol2_files, ligand_files, 
                 slopes_file = '', excel_sheet = '', cofactor ='', 
                 gpu_vina = False, result_path = 'Results', receptors = '',
                 NADP_cofactor = False, metal_containing = False, align = False, output_formate = 'pdbqt', 
                 show_plots = False, plot = False, 
                 center =[-12.951,-44.891,4.451], size = [25,25,25], 
                 #nr of states
                 num_modes = 9, 
                 #patients
                 exhaustiveness = 64, 
                 #How different do the states have to be to be "different"
                 energy_range = 3):
        
        self.working_dir = working_dir
        self.ligand_files = ligand_files
        self.slopes_file = slopes_file
        self.excel_sheet = excel_sheet
        self.mol2_files = mol2_files
        self.cofactor = cofactor
        self.plot = plot
        self.gpu_vina = gpu_vina
        self.NADP_cofactor = NADP_cofactor
        self.metal_containing = metal_containing #!!!!
        self.align = align
        self.output_formate = output_formate
        self.show_plots = show_plots
        self.center = center
        self.size = size
        self.num_modes = num_modes
        self.exhaustiveness = exhaustiveness
        self.energy_range = energy_range
        self.result_path = result_path



config = config(working_dir, ligand_files, slopes_file, excel_sheet, mol2_files)
#config.gpu_vina = True #set to True if you want to use GPU Vina
config.NADP_cofactor = False #set to True if you want to use NADP cofactor

In [1]:
#import all modules needed

from rdkit.Chem.PandasTools import LoadSDF
from rdkit.Chem import Descriptors
from rdkit import Chem
import os
import pandas as pd
import subprocess

from biopandas.pdb import PandasPdb
from biopandas.mol2 import split_multimol2
from biopandas.mol2 import PandasMol2
#pdmol = PandasMol2()
import shutil
import pickle

import multiprocessing
import matplotlib.pyplot as plt
from natsort import natsorted

import glob
import time
from tqdm.auto import tqdm

from biopandas.mol2 import PandasMol2
import numpy

import pymol
from pymol.cgo import *


from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, QED
from rdkit.Chem.Draw import IPythonConsole
import numpy as np
#We suppres stdout from invalid smiles and validations
from rdkit import rdBase
rdBase.DisableLog('rdApp.*')

Chem.PandasTools.RenderImagesInAllDataFrames(images=True)

'''Machine learning approach based on df_activity'''

# Load libraries
from pandas.plotting import scatter_matrix
from matplotlib import pyplot

#prepare data
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

#model metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#different models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier

# Packages for ML
import sys
print('Python: {}'.format(sys.version))
import scipy
print('scipy: {}'.format(scipy.__version__))

print('numpy: {}'.format(numpy.__version__))
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

pd.options.mode.chained_assignment = None  # supress warnings

[16:00:07] Enabling RDKit 2019.09.3 jupyter extensions
RDKit WARNING: [16:00:07] Enabling RDKit 2019.09.3 jupyter extensions
/home/cewinharhar/anaconda3/envs/docking/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python: 3.8.16 (default, Jan 17 2023, 23:13:24) 
[GCC 11.2.0]
scipy: 1.5.3
numpy: 1.22.3
matplotlib: 3.7.0
sklearn: 1.1.1


In [32]:
def log(log_file_path,message):
    with open(log_file_path, "a+") as f:
        f.write(message + "\n")
    f.close()

#if list with smiles. smiles are converted into dictionary with smiles as key and molecule as value. 
# In case of mol2 files, get all mol2 files in folder and convert them into smiles and create df_ligands
def mol2_to_df(mol2_files):
    '''Function to convert mol2 files to df_ligand and create 3-dimensional sdf files'''
    mol2_files_to_convert = ''
    for mol2_file in mol2_files:
        mol2_files_to_convert += mol2_file+' '
    os.system(f"obabel {mol2_files_to_convert} -O ligands.smiles")
    f=open('ligands.smiles','r')
    lines=f.readlines()
    molecule_library ={}
    for line in lines:
        key = line.split(sep='\t')[1].strip('\n')
        item = line.split(sep='\t')[0]
        molecule_library[key] = item

    df_ligands = pd.DataFrame(columns=['ligand_names','molecule_smiles'])
    for key, value in molecule_library.items():
        df_ligands = df_ligands.append({'ligand_names':key,'molecule_smiles':value},ignore_index=True)


        #generate molecules as sdf file
    mols = [Chem.MolFromSmiles(x) for x in df_ligands['molecule_smiles']]
    with Chem.SDWriter('ligands.sdf') as w:
        for idx, m in enumerate(mols):
            w.write(m)

    if not glob.glob('ligands3D.sdf'): #Return a list of paths matching a pathname pattern.
        generate_3d_sdf = f"obabel ligands.sdf -O ligands3D.sdf --gen3D -aa"
        ps = subprocess.Popen(generate_3d_sdf,shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        stdout,stderr = ps.communicate()
        print(stdout,stderr)
    return df_ligands
    
    
def smiles_to_dict(file):
    molecule_library = {}
    f=open(file)
    lines=f.readlines()
    for line in lines:
        key = line.split(sep='\t')[1].strip('.log\n')
        item = line.split(sep='\t')[0]
        molecule_library[key] = item
    return molecule_library


    
#some helper functions to get descriptors and molecules based on nomenclature
def get_descriptors(mol):
    """ only get from substrate, just chemo phsyical infos"""
    
    logp = Descriptors.MolLogP(mol)
    tpsa = Descriptors.TPSA(mol)
    molwt = Descriptors.ExactMolWt(mol)
    hba = rdMolDescriptors.CalcNumHBA(mol)
    hbd = rdMolDescriptors.CalcNumHBD(mol)
    qed = QED.qed(mol)
    
    slogP = Descriptors.SlogP_VSA1(mol)
# Calculate fingerprints
    fp = AllChem.GetMorganFingerprintAsBitVect(mol,2, nBits=2048)
    ecfp4 = np.zeros((2048,))
    DataStructs.ConvertToNumpyArray(fp, ecfp4)

    return (logp, tpsa, molwt, qed,hba,hbd)


def make_molecule(smiles):
    #make 3d molec from smiles
    #sanitizer 
    try:
        mol = Chem.MolFromSmiles(smiles)
        m = Chem.MolToInchi(mol)
        mol_cured = Chem.MolFromInchi(m)

    except:
        print('Error with smile: ',smiles)

    return mol_cured

""" def prepare_ligands_from_experimental_data(excel_path, excel_tab, molecule_library):
    
    df_slopes = pd.read_excel(excel_path, excel_tab)
    df_slopes['enzyme_reaction'] = df_slopes.apply(lambda row: row.protein + '_'+ row.nitriles,axis=1)
    df_slopes['ligand_names'] = df_slopes['nitriles']
    df_slopes['molecule_smiles'] = df_slopes['nitriles'].map(molecule_library)
    df_slopes['molecule'] = df_slopes.apply(lambda row: make_molecule(row.molecule_smiles), axis=1)
    df_slopes['logp'], df_slopes['tpsa'], df_slopes['molwt'], df_slopes['qed'], df_slopes['hba'], df_slopes['hbd'] = zip(*df_slopes['molecule'].map(get_descriptors))
    
    df_slopes['active'] = df_slopes['activity'].apply(lambda x:'inactive' if x < 2 else 'active')  # active if conversion above 2mM

    df_ligands = df_slopes.drop_duplicates(subset=['molecule_smiles'], keep='last')
    df_ligands['ligand_names']=df_slopes['nitriles']

    #generate molecules as sdf file
    mols = [Chem.MolFromSmiles(x) for x in df_ligands['molecule_smiles']]
    with Chem.SDWriter('ligands.sdf') as w:
        for idx, m in enumerate(mols):
            w.write(m)

    if not glob.glob('ligands3D.sdf'):
        generate_3d_sdf = f"obabel ligands.sdf -O ligands3D.sdf --gen3D -aa"
        ps = subprocess.Popen(generate_3d_sdf,shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        stdout,stderr = ps.communicate()
        print(stdout,stderr)

    else:
        print('3D high energy intermediates do already exist. Delete file if you want to generate them new')
    
    
    df_ligands = df_ligands[['ligand_names','enzyme_reaction']].copy()
    
    
    return df_slopes, df_ligands """

def find_box_center(filename, cent):
    #identify heteroatoms and take first chain_id, identify c4n only possible in case of NAD(maybe) or NADP needs to be adjusted according to pdb
    
    #enzyme_files = [file for file in os.listdir() if file.endswith('.pdb')]

    #!!!! WITH aKGD works with metal = true
        #check config

    enz = PandasPdb().read_pdb(filename)
    try:
        cofactors=enz.df['HETATM']
        gb = cofactors.groupby('chain_id')
        cofactor1 = [gb.get_group(x) for x in gb.groups][0]
        c5n = cofactor1[cofactor1.atom_name == 'C5N']
        if c5n.empty:
            c5n = cofactor1[cofactor1.atom_name == 'C5']

        ref_point1 = (c5n.x_coord.values[0], c5n.y_coord.values[0],c5n.z_coord.values[0])
        return list(ref_point1)
    except:
        print('could not find cofactor in: ', filename)
        return cent

def prepare_docking_files():

    docking_path = config.working_dir

    if os.path.exists(docking_path):
        os.chdir(docking_path)  # we are now in the docking folder
        print('Docking folder:',os.getcwd())
    else:
        print('no valid docking folder')


    if not os.path.exists('Experiment_log_file.txt'):

        log_file_path = os.path.join(docking_path,"Experiment_log_file.txt")
        print(log_file_path)
        log(log_file_path,message)
        log(log_file_path,'substances_db: '+ str(config.mol2_files))
        log(log_file_path,'center_coordinates: '+str(config.center))
        log(log_file_path,'box size in angström: '+str(config.size))
        log(log_file_path,'num_modes: '+str(config.num_modes))
        log(log_file_path,'exhaustiveness: '+str(config.exhaustiveness))
        log(log_file_path,'energy_range: '+str(config.energy_range))
        log(log_file_path,'output_formate: '+config.output_formate)
        log(log_file_path,'metal containing: '+str(config.metal_containing))

    else:
        print('Log file already there!')



In [4]:
def receptor_list(formate):
    all_receptors=[]
    for file in os.listdir(config.working_dir):
        if file.endswith(formate):
            all_receptors.append(file)
    return all_receptors

def prepare_receptor(filename, new_filename):
    #Enzyme to pdbqt(includes charge & atom type) file
    prepare_receptor=f'/home/jupyter/ADFRsuite-1.0/bin/prepare_receptor -r {filename} -o {new_filename}.pdbqt -A hydrogens -v -U nphs_lps_waters'
    ps = subprocess.Popen([prepare_receptor],shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    stdout,stderr = ps.communicate()
    #print(stdout)
    if stderr != None:
        print(stderr,'error')
    
    return


def receptor(filename):

    #print(filename, 'is converted to pdbqt')
    new_filename=filename[:-4]

    cmd.reinitialize() 
    cmd.load(filename)
    area=cmd.get_area()
    
    center=config.center  # overwrite variable to use the same coordinates in all enzymes
    
    if config.metal_containing:
        print('Metal coordinates identified')
        cmd.select('metals')
        xyz=cmd.get_coords('sele')
        cen=xyz.tolist()
        center=cen[0]
        
    if config.NADP_cofactor:
        center=find_box_center(filename, config.center)
        
    cmd.reinitialize()
    
    prepare_receptor(filename,new_filename)

    return center


def prepare_all_receptors(nrCPU = 8):

    """
    nrCPU = nr of processors oc cpu
    """
    
    path=os.getcwd()
    
    all_receptors=receptor_list('.pdb')
    #enter here check for pdbqt files if len found eqaul to pdb don't do the below stuff and return bc from for loop

    if (len(receptor_list('.pdbqt')) < len(all_receptors)):
        print('pdb to pdbqt conversion of receptors')
        
        if config.NADP_cofactor:
            print('Trying to identify atom C5N of cofactor NADP')
            
        with multiprocessing.Pool(processes=nrCPU) as pool:
            bc = pool.starmap(receptor,[[x] for x in all_receptors])
        
        with open("box_centers", "wb") as fp:   # Pickling
            pickle.dump(bc, fp)

        print('conversion finished')
        
    else:
        print('receptor preparation skipped. Already done.')
        with open("box_centers", "rb") as fp:   # Unpickling
            bc = pickle.load(fp)

    print(bc)  # possible box coordinates 
    return (all_receptors,bc)



def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


def analyse_affinities_all(receptor,rec_path,rec_sub_path, df_all):
    """
    Result of VINA gives text log: extract information function
    """
    error=0
    for file in os.listdir(rec_path):
        if file.endswith(".log"):
            f=open(rec_sub_path+file)
            lines=f.readlines()
            parts=file.split(sep='_')


            
            try:
                if config.cofactor == '':
                    en = []
                    en1 = []
                    en2 = []
                    en3 = []
                    en4 = []
         
                    for line in lines:  # only include first docking pose
                        if line.startswith('Estimated Free Energy of Binding  '):
                            en.append(float(line.split()[-3]))
                            
                        if line.startswith('(1)'):
                            en1.append(float(line.split()[-2]))
                        if line.startswith('(2)'):
                            en2.append(float(line.split()[-2]))
                        if line.startswith('(3)'):
                            en3.append(float(line.split()[-2]))
                        if line.startswith('(4)'):
                            en4.append(float(line.split()[-2]))
                            

                    df_energies= pd.DataFrame([en+en1+en2+en3+en4], columns=['Energies_' + str(x+1) for x in range(len(en))]+['Energies1_' + str(x+1) for x in range(len(en1))]
                                              +['Energies2_' + str(x+1) for x in range(len(en2))]+['Energies3_' + str(x+1) for x in range(len(en3))]+['Energies4_' + str(x+1) for x in range(len(en4))]).astype(float)
    

                else:
                    print('needs to be done')
                
                
                if False == (df_energies.dtypes == np.float64).all():
                    error+=1
                    print('this log file looks strange',file)
                    
            except:
                error+=1
                print("log error", file)
                en=0
                continue
            
            
            df_energies['res_out'] = os.path.join(rec_path,file[:-8]+'_out.'+config.output_formate)
            
            df_energies['ligand_names'] = parts[-2]
            #split of log file using underline but ligand defined with underline as well
            
            df_energies['Enzyme_group'] = parts[0]
            df_energies['Log_files'] = file
            df_energies['res_path'] = rec_sub_path
            df_energies['Enzyme_ligand'] = parts[0]+'_'+parts[-3]
            
           

            f.close()
            
            df_all = pd.concat([df_all, df_energies], sort=False)
            

    

    df_all = df_all.reset_index(drop=True)

    if config.plot:
        substances_db = str(config.mol2_files)
    
        fig = plt.figure(figsize=(20,15))
        
        ax1 = fig.add_subplot()
        ax1.set_ylabel('binding affinity / (-kcal/mol)')
        ax1.set_title('Docking of '+receptor[:-6]+' with '+substances_db[:-6])
        

        heights = (-1)*(df_all.Energies_1)
        bars = df_all.Enzyme_group + '_' + df_all.ligand_names
        y_pos = range(len(bars))
        barlist=plt.bar(y_pos, heights)
        
        df_all['max'] = False
        

        
        ind = 0
        #ind = df['Energies'].argmin()
        barlist[ind].set_color('r')
        df_all['max'][ind]=True
        

        
        
        # Rotation of the bars names
        plt.xticks(y_pos, bars, rotation=90)
        
        plt.savefig(rec_sub_path+receptor[:-6]+'_results.png')
        plt.tight_layout()
        if config.show_plots == False:
            plt.close()
        
        
    df_all.to_excel(rec_sub_path+receptor[:-6]+'_results.xlsx')

    return df_all

In [5]:
# NOT NEEDED

def get_activity(activity,control,a,k,e,sd2,sd3,sd4):
    # get only_amine, only_ketone or only_enzyme value if available and check which one is the highest value
    # use the highest value and the standardeviation if availble to calculate LOQ and check if activity is above that value
    
    control_high=min(a,k,e)
    
    if np.isnan(control_high):
        if abs(activity)-abs(control)*4 > 0:
            return 'pos'
        else:
            return 'neg'
    else:
        
        if a==control_high:
            sd=sd2
        if k== control_high:
            sd=sd3
        if e== control_high:
            sd=sd4
        
        
        if (abs(activity)-abs(control_high)-abs(sd)*10) > 0:
            return 'pos'
        else:
            return 'neg'

    
    #what to do if there is no control values?
    #maybe amine contributes the most to background get closest amine based reaction and do the rest at the beggining just skip these values and label them unkown

def prepare_ligands(substances_db, result_path, df_ligands, receptors):
    """
    SDF file can save multiple substrates in 3D (list of mol2 files)
    """
    if not glob.glob(config.result_path+'/'+'*.sdf'):
    
        #prepare_ligand=f'/home/meeb/ADFRsuite-1.0/bin/prepare_ligand -r {ligand_name} -o {sub_path}{ligand_name}.pdbqt -A hydrogens -v -U nphs_lps_waters'

        os.system(f"obabel {substances_db} -O {config.result_path+'/'}Ligand.sdf -m")


        lig_sdf=[]
        lig_sdf_new=[]
        for file in natsorted(glob.glob(os.path.join(result_path,'*.sdf'))):
            lig_sdf.append(file)
        print(len(lig_sdf))
 

        for idx,n in enumerate(lig_sdf):
            lig_sdf_new.append(os.path.join(result_path,list(df_ligands.ligand_names)[idx])+'.sdf')
            os.rename(n,os.path.join(result_path,list(df_ligands.ligand_names)[idx])+'.sdf')
        
        for lig in lig_sdf_new:
            prepare_ligand=f'/home/jupyter/Meeko/scripts/mk_prepare_ligand.py -i {lig}'

            ps = subprocess.Popen([prepare_ligand],shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
            stdout,stderr = ps.communicate()
            if len(stdout) !=0:
                print(stdout)
            if stderr != None:
                print(stderr,'error')

        all_lig_files=[]
        full_paths = []
        for file in os.listdir(result_path):
            if file.endswith('.pdbqt') and (file not in receptors):
                full_paths.append(os.path.join(result_path, file))
                all_lig_files.append(file)
        all_lig_files=natsorted(all_lig_files)

        #shortcut to log the molecule information (name etc)
        f = open(config.result_path+'/'+"Ligand.txt", "a+")
        f.writelines(["%s\n" % item for item in all_lig_files])
        f.close()
        print(len(all_lig_files))

        return all_lig_files
    
    else:
        print('Ligand files do already exist')
    
def prepare_cofactor():
    """
    ONLY NEEDED IF SIMULTANIOUS DOCKING OF SUBSTRATE AND COFACTOR
    
    """
    second_ligand = config.cofactor[:-5]+'.pdbqt'
    prepare_cofactor=f'/home/jupyter/Meeko/scripts/mk_prepare_ligand.py -i {config.cofactor} -o {second_ligand}'
    
    print(' CHECK AGAIN PLS')

    ps = subprocess.Popen([prepare_cofactor],shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    stdout,stderr = ps.communicate()
    #print(stdout)
    if stderr != None:
        print(stderr,'error')
    
    return second_ligand

In [7]:
#execute vina.exe with variables

def vina_docking(lig,file_name,receptor_v,center_v):
    """ if verbosity is changed to 2 (only non gpu vina) output is affinity + other params (might be usable for ml approach)"""
    
    center_x=center_v[0]
    center_y=center_v[1]
    center_z=center_v[2]

    (size_x, size_y, size_z) = config.size
    if config.cofactor != '':
        second_ligand = config.cofactor[:-5]+'.pdbqt'
    else:
        second_ligand = ''

    
    if config.gpu_vina:
        file_name = working_dir+file_name
        lig = working_dir+lig
        receptor_v = working_dir+receptor_v
        vina_docking=f"/home/jupyter/vina_gpu/Vina-GPU/Vina-GPU --thread 8000 --receptor {receptor_v} --ligand {lig} {second_ligand} --seed 42 --center_x {center_x} --center_y {center_y} --out {file_name}_out.{config.output_formate} --center_z {center_z} --size_x {size_x} --size_y {size_y} --size_z {size_z} --num_modes {config.num_modes} --energy_range {config.energy_range}"
    else:
        vina_docking=f"/home/jupyter/vina/vina_1.2.3_linux_x86_64 --verbosity 2 --receptor {receptor_v} --ligand {lig} {second_ligand} --seed 42 --center_x {center_x} --center_y {center_y} --out {file_name}_out.{config.output_formate} --center_z {center_z} --size_x {size_x} --size_y {size_y} --size_z {size_z} --num_modes {config.num_modes} --exhaustiveness {config.exhaustiveness} --energy_range {config.energy_range}"


    log_file = f"{file_name}_log.log"
    #print(vina_docking)
    ps = subprocess.Popen([vina_docking],shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    stdout,stderr = ps.communicate()
    os.chdir(working_dir)
    log(log_file,stdout.decode())
    if stderr != None:  
        log(log_file,stderr.decode())
        


In [6]:
#FUNCTION TO SUMMARIZE ALL FUNCTIONS AND USE DOCKING

def virtual_screen(receptor, center):
    sub_path = config.result_path+'/'

    rec_path=os.path.join(sub_path, receptor[:-6])
    rec_sub_path=sub_path+receptor[:-6]+'/'

    if not os.path.exists(rec_path):
        os.makedirs(rec_path)

    with open(sub_path+"Ligand.txt","r") as f:

        lines = f.readlines()
        print('number of ligands',len(lines))
        f.close()

    fn=[]
    lig=[]
    for i in lines:
        i = i.rstrip("\n")
        file_name=i.split(".")[0]  # give file name without formate ending
        file_name=receptor[:-6]+'_'+file_name
        file_name=rec_sub_path+file_name  # log file into specific receptor result folder
        i = sub_path+i  # take ligand from sub_path rather than result file for receptor
        lig.append(i)
        fn.append(file_name)

                
    receptor_v = [receptor]*len(lig) #preparing receptor as list for zipping
    center_v = [center]*len(lig) #preparing center as list for zipping
    if config.gpu_vina:
        os.chdir('/home/jupyter/vina_gpu/Vina-GPU')
    
    if len(lig) > len(config.receptors):
        print('more ligands than receptors')
        with multiprocessing.Pool(processes=int(50/config.exhaustiveness)) as pool: #limit multiprocessing to 50 cores depending on exhaustiveness
                pool.starmap(vina_docking,tqdm(zip(lig,fn,receptor_v,center_v), total = len(lig)))  #tqdm and total = len(lig) for progressbar
        

    else:
        print('more receptors than ligands')

        for i in range(len(lig)):
            vina_docking(lig[i],fn[i],receptor_v[i],center_v[i])
    
    if config.gpu_vina:
        os.chdir(working_dir)

    df =pd.DataFrame()
    if config.gpu_vina:
        return df
    else:
        df=analyse_affinities_all(receptor,rec_path,rec_sub_path,df)
        return df

# DF = MOST IMPORTANT DF

def delete_all_pdbqt_files():
    for file in os.listdir():
        if file.endswith('.pdbqt'):
            os.remove(file)
    print('all pdbqt files deleted')
#delete_all_pdbqt_files()

#function to play a sound
from IPython.display import Audio, display

def allDone():
    display(Audio('/home/jingle-bell-1.mp3', autoplay=True))

In [7]:
""" def plot_binding_affinity(df_xlsx,substances_db,destination):

    fig = plt.figure(figsize=(35,15))

    ax1 = fig.add_subplot()
    ax1.set_ylabel('binding affinity / (-kcal/mol)')
    ax1.set_title('Docking of all receptors with '+substances_db[:-4])

    heights = (-1)*(df_xlsx.Energies_1)
    bars = df_xlsx.Enzyme_ligand
    y_pos = range(len(bars))
    barlist=plt.bar(y_pos, heights)

    print('for '+substances_db[:-4]+' the best affinity is '+str(df_xlsx.Energies_1.min())+' kcal/mol')


    #color best affinities in red
    best=df_xlsx[df_xlsx.Energies_1 ==df_xlsx.Energies_1.min()]
    best_list=list(best.index)

    for ind in best_list:
        barlist[ind].set_color('r')

    # Rotation of the bars names
    plt.xticks(y_pos, bars, rotation=90)
    plt.tight_layout()

    plt.savefig(os.path.join(destination,'All_results.png'))
    if config.show_plots == False:
        plt.close()
    return best """

def summarize_results(res1, substances_db):
    df_xlsx=pd.DataFrame()
    for i in range(len(res1)):
        #res1[i]['Enzyme_group']=i
        df_xlsx=df_xlsx.append(res1[i])

    df_xlsx['out']=df_xlsx['res_path']+df_xlsx['Log_files']
    try:
        df_xlsx['enzyme_reaction'] = df_xlsx.apply(lambda row: row.Enzyme_group + '_'+ row.ligand_names,axis=1)
    except:
        print('no enzyme groups or ligand names')
    df_xlsx=df_xlsx.reset_index()

    # Destination path create summary folder which harbors all the results summarized
    path = os.getcwd()
    destination =os.path.join(path,'Results_summary')
    if not os.path.exists(destination):
        os.makedirs(destination)

    best=plot_binding_affinity(df_xlsx, substances_db,destination)
    df_xlsx.to_excel(os.path.join(destination,'Overview_binding_affinity.xlsx'))

    #copy best files to Results_summary folder

    for i in range(len(best)):
        log_file = os.path.join(path, best.iloc[i]['out'])
        log_file_dest = os.path.join(destination, best.iloc[i]['Log_files'])

        out_file = os.path.join(path, best.iloc[i]['out'][:-8]+'_out.pdbqt')  # maybe add into result file instead
        out_file_dest = os.path.join(destination, best.iloc[i]['Log_files'][:-8]+'_out.pdbqt')  # maybe add into result file instead
        try:
            shutil.copyfile(log_file, log_file_dest)  # copy from source to destination
            shutil.copyfile(out_file, out_file_dest)  # copy from source to destination
        except:
            print('copy error')
            
            
        #convert all result files into mol2

    res_outs=df_xlsx['res_out']
    res_mols = [[]]*len(res_outs)
    
    mol_files = glob.glob("Results/**/*.mol2", recursive=True)
    
    for i in range(len(res_outs)):
        
        if len(mol_files) < len(res_outs):
            pdbqt_to_mol = f"obabel {res_outs[i]} -O {res_outs[i][:-10]}.mol2"

            ps = subprocess.Popen(pdbqt_to_mol,shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
            stdout,stderr = ps.communicate()
        

        res_mols[i]=res_outs[i][:-10]+".mol2"

    df_xlsx['res_mols']=res_mols
    #df_xlsx['Enzyme_group'] = df_xlsx.apply(lambda row: row.Ligand.split(sep='_')[0],axis = 1)
    print('adding no distancese')
    #df_xlsx = add_distances(df_xlsx)
    
    return df_xlsx

In [8]:
def measure_angle_distance_multi(res_file,func_group):
    ''' Input res_file containing enzyme and ligand docked. Function is loading first the corresponding Enzyme_file and is searching the Cofactor NADP as ref point. Afterwards measurement is carried out for each state'''
    distances = []
    angles = []
    
    enzyme_file = res_file.split(sep='/')[1]+'.pdb'
    
    enz = PandasPdb().read_pdb(enzyme_file)

    cofactors=enz.df['HETATM']
    gb = cofactors.groupby('chain_id')
    cofactor1 = [gb.get_group(x) for x in gb.groups][0]

    c4n = cofactor1[cofactor1.atom_name == 'C4N']
    c5n = cofactor1[cofactor1.atom_name == 'C5N']
    

    ref_point1 = (c4n.x_coord.values[0], c4n.y_coord.values[0],c4n.z_coord.values[0])
    ref_point2 = (c5n.x_coord.values[0], c5n.y_coord.values[0],c5n.z_coord.values[0])
    

    if len(ref_point1) == 0 or len(ref_point2) == 0:
        print('couldnt find the cofactor')
    
    for mol2 in split_multimol2(res_file):
        pmol = PandasMol2().read_mol2_from_list(mol2_lines=mol2[1], mol2_code=mol2[0])
    
        if func_group == 'keto':
            f= pmol.df[pmol.df['atom_type'] == 'O.2']
        elif func_group == 'sp2C':
            f= pmol.df[pmol.df['atom_type'] == 'C.2']

        elif func_group == 'sp3C':
            f= pmol.df[pmol.df['atom_type'] == 'C.3']

        elif func_group == 'spC':
            f= pmol.df[pmol.df['atom_type'] == 'C.1']

        elif func_group == 'N':
            f= pmol.df[pmol.df['atom_type'] == 'N.2']
            if len(f)==0:
                f = pmol.df[pmol.df['atom_type'] == 'N.3']
                if len(f)==0:
                    f = pmol.df[pmol.df['atom_type'] == 'N.pl3']
                    # print('no sp3 nor sp2 detected, only planar N.pl3', res_file)
                    # check N.pl3 there should be a work around with meeko
        else:
            print('functional group not found')
            
        try:
            # it might be important to perform the calcuations using the correct index check again
        
            x_f,y_f,z_f = f.iloc[0]['x'],f.iloc[0]['y'],f.iloc[0]['z']  # get coordinates of certain atom point
            atom=np.array([x_f,y_f,z_f])
            # calculating distance
            distance = np.linalg.norm(atom - ref_point1)
            # calculating angle
            ba = atom - ref_point1
            bc = atom - ref_point2

            cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))

            angle = np.arccos(cosine_angle)

            distances.append(distance)
            angles.append(angle)
                
        except:
            print(res_file,'measure distance error', 'functional_group',func_group)
            distances = 'error'
            angles = 'error'

    return pd.Series([distances,angles])

In [9]:
def add_distances(df_xlsx):

    df_xlsx[['distance_His','angle_His']] = df_xlsx.apply(lambda x: measure_angle_distance_multi(x["res_mols"],'N'),axis=1)
    df_xlsx[['distance_His_' + str(x+1) for x in range(9)]] = pd.DataFrame(df_xlsx.distance_His.tolist(), index=df_xlsx.index)
    df_xlsx[['angle_His_' + str(x+1) for x in range(9)]] = pd.DataFrame(df_xlsx.angle_His.tolist(), index=df_xlsx.index)
    
    print('His distances/angles added')
    
    df_xlsx[['distance_NADP','angle_NADP']] = df_xlsx.apply(lambda x: measure_angle_distance_multi(x["res_mols"],'N'),axis=1)
    df_xlsx[['distance_NADP_' + str(x+1) for x in range(9)]] = pd.DataFrame(df_xlsx.distance_NADP.tolist(), index=df_xlsx.index)
    df_xlsx[['angle_NADP_' + str(x+1) for x in range(9)]] = pd.DataFrame(df_xlsx.angle_NADP.tolist(), index=df_xlsx.index)
    
    print('NADP distances/angles added')
   
    return df_xlsx

In [12]:
#this block is only required if post analysis is needed


def post_processing():
    """
    Reload data from log
    """
    path=os.getcwd()
    
    def receptor_list(formate):
        all_receptors=[]
        for file in os.listdir(path):
            if file.endswith(formate):
                all_receptors.append(file)
        return all_receptors

    def post_analysis(receptor):
        sub_path = config.result_path+'/'
        rec_path=os.path.join(sub_path, receptor[:-6])
        rec_sub_path=sub_path+receptor[:-6]+'/'
        df=pd.DataFrame()
        df=analyse_affinities_all(receptor,rec_path,rec_sub_path, df)
        return df


    all_receptors=receptor_list('.pdbqt')
  
    res1=[[]]*len(all_receptors)
    for i in range(len(all_receptors)):
        res1[i]=post_analysis(all_receptors[i])
    print('data loaded')
    return res1

def align_pdbs():
    """
    Align enzymes (normally not usable)
    """
    if not os.path.exists('original_pdbs'):
        os.makedirs('original_pdbs')
    all_pdb = [pdb for pdb in os.listdir() if pdb.endswith('.pdb') and not pdb.endswith('aligned.pdb')]
    t=all_pdb[1:]  # get all pdbs without first one
    cmd.reinitialize()
    for pdb in all_pdb:
        
        cmd.load(pdb)
        
    for pdb in t:
        cmd.align(pdb[:-4],all_pdb[0][:-4])
    for pdb in all_pdb:
        
        os.system(f'mv {pdb} original_pdbs/{pdb}')
        cmd.save(pdb[:-4]+'_aligned.pdb', selection=pdb[:-4])
        
    print(len(all_pdb),' pdbs aligned and original moved to orignal_pdb folder')

In [13]:
#Docking
#move all homology models to the current working directory

# ''' All variables needed which can be modified by the user '''

# #enter path to docking folder containing all receptors as pdb files including ligand library as sdf
# docking_path=working_dir  # no spaces allowed
# os.chdir(working_dir)

# #substances_db='ligands3D.sdf'  # substance database either as sdf or pdbqt file currently it seems there is a size limits needs to be checked
# cofactor=''


# #if multiple pdb files exist and they are homologs to each other it might make sense to align them before docking to set the box coordinates at the same postion in all cases



# center_x,center_y,center_z= -12.951,-44.891,4.451 # center of the box not used if NADP is present
# cent=[center_x,center_y,center_z]

# size_x,size_y,size_z=25,25,25  # size of the box. Makes sense to keep small the same for all boxes seems okay maybe adjust if substrate too big
# size=[size_x,size_y,size_z]


# num_modes = 9  # number of docking modes if 1 is chosen might cause problem default=9
# exhaustiveness = 8  # exhaustiveness default=8
# energy_range = 3  # energy range implies difference between docking modes default=3

# metal_containing = False  # true if enzyme contains metal in the active site

# NADP_cofactor = True
# # true if enzyme contains NADP as cofactor which should act as center
# show_plots = False
# output_formate='pdbqt'


## THE MAGIC HAPPENS HERE


In [14]:
#main function for docking start all
start = time.time()

os.chdir(config.working_dir) #move all homology models to the current working directory containing all receptors as pdb files including ligand library as sdf
(center_x, center_y, center_z) = config.center
if config.align:
    #if multiple pdb files exist and they are homologs to each other it might make sense to align them before docking to set the box coordinates at the same postion in all cases
    #if true this will be done using pymol
    align_pdbs()
else:
    print('no alignment performed')

if ligand_files.endswith('.smiles'):
    molecule_library = smiles_to_dict(ligand_files)
elif ligand_files.endswith('.sdf'):
    print('sdf file conversion function not done yet')
elif ligand_files == '':
    print('no ligand library provided as smiles or sdf file')

if len([file for file in mol2_files if file.endswith('.mol2')]) >= 0:
    df_ligands = mol2_to_df(mol2_files)
    substances_db = 'ligands3D.sdf'
else:
    print('no mol2 files provided')

if slopes_file != '':
    (df_slopes,df_ligands) = prepare_ligands_from_experimental_data(slopes_file, excel_sheet)

prepare_docking_files()


result_path = os.path.join(config.working_dir,config.result_path)
#sub_path = os.path.basename(result_path)+'/'


all_receptors,bc = prepare_all_receptors()
    
#receptors=receptor_list('.pdbqt') the order seems to be different therefore code below
receptors=[]
for i in all_receptors:
    re=i[:-4]+'.pdbqt'
    receptors.append(re)
config.receptors = receptors
print('receptors',receptors)
print('length receptors',len(receptors))
print('ligand database',substances_db)

res1=len(receptors)*[[]]

print(result_path)
if not os.path.exists(result_path):
    os.makedirs(result_path)

        

    if config.cofactor != '':
        if config.cofactor.endswith('.mol2'):
            second_ligand=prepare_cofactor()
        else:
            print('cofactor formate wrong. needs to be mol2')
    else:
        second_ligand = ''

    #new way of preparing ligands using meeko approach
    ligands =prepare_ligands(substances_db, result_path, df_ligands,receptors)
    
    if len(ligands) > len(receptors):
        print('more ligands than receptors')
        for i in tqdm(range(len(receptors)), total=len(receptors)): 
            res1[i]=virtual_screen(receptors[i],bc[i])

    else:
        print('more receptors than ligands')


        if config.gpu_vina:
            
            for i in tqdm(range(len(receptors)), total=len(receptors)):
                res1[i]=virtual_screen(receptors[i],bc[i])
            
        else:
            with multiprocessing.Pool(processes=4) as pool: #limit multiprocessing to 50 cores
                for result in pool.starmap(virtual_screen,tqdm(zip(receptors,bc), total = len(receptors))):
                    res1.append(result)

    

    print('Multidocking finished !!!!')
    
    if config.gpu_vina:
        print('GPU Vina used data not summarized')
        
    else:
        df_xlsx = summarize_results(res1, substances_db)

        with open("df_xlsx.pkl", "wb") as fp:   # Pickling
                pickle.dump(df_xlsx, fp)

    print('It took', time.time()-start, 'seconds.')


else:
    if os.path.exists('df_xlsx.pkl'):
        with open("df_xlsx.pkl", "rb") as fp:   # Unpickling
            df_xlsx = pickle.load(fp)
    else:
        res1 = post_processing()
        df_xlsx = summarize_results(res1, substances_db)
        with open("df_xlsx.pkl", "wb") as fp:   # Pickling
            pickle.dump(df_xlsx, fp)
    
    print('It took', time.time()-start, 'seconds. To re-load results.')
    print('\n Docking already done. Do you want to redo it? Delete Results folder.')

allDone()

no alignment performed
no ligand library provided as smiles or sdf file
Docking folder: /home/pyrosetta/EnzyDel1/variants_wo_ligand
Log file already there!
receptor preparation skipped. Already done.
[[5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30.517], [5.415, -25.663, 30

1 molecule converted
1 molecule converted


1
more receptors than ligands


  0%|          | 0/500 [00:00<?, ?it/s]

number of ligandsnumber of ligandsnumber of ligands  number of ligands1 1
1 more receptors than ligands

1
more receptors than ligands

more receptors than ligandsmore receptors than ligands

number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than ligands
number of ligands 1
more receptors than 

In [35]:
m = Chem.MolFromSmiles("CC(=O)CCc1ccccc1")

with open("test.mol2", "w") as file:
    file.write(Chem.MolToMolBlock(m))


In [4]:
m = Chem.MolFromSmiles("CC(=O)CCc1ccccc1")
Chem.MolToInchi(m)

'InChI=1S/C10H12O/c1-9(11)7-8-10-5-3-2-4-6-10/h2-6H,7-8H2,1H3'

In [25]:
Chem.Draw.MolToFile(m, "testImage.png")

In [11]:
from src.dockingHelpers.smiles2Mol import make_molecule

ModuleNotFoundError: No module named 'src.dockingHelpers'

In [10]:
os.chdir("/home/cewinharhar/GITHUB/gaesp/src")
os.getcwd()

'/home/cewinharhar/GITHUB/gaesp/src'

In [14]:
def make_molecule(smiles):
    #make 3d molec from smiles
    #sanitizer 
    try:
        mol = Chem.MolFromSmiles(smiles)
        m = Chem.MolToInchi(mol)
        mol_cured = Chem.MolFromInchi(m)

    except:
        print('Error with smile: ',smiles)

    return mol_cured

In [22]:
mu = make_molecule("CC(=O)CCc1ccccc1")
Descriptors.MolLogP(mu)

2.2081999999999997

In [21]:
with open("mu.mol", "w") as file:
    file.write(Chem.MolToMolBlock(mu))